In [112]:

import numpy as np
import pandas as pd


from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.linear_model import SGDClassifier
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix



import datetime
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV

In [2]:
train_df = pd.read_csv(r'C:\Users\ILYA\Downloads\train_sessions.csv (1)\train_sessions.csv', index_col='session_id')
#D:\Загрузки\train_sessions.csv\train_sessions.csv     из красного 
# C:\Users\ILYA\Downloads\train_sessions.csv (1)\train_sessions.csv    из серого

In [3]:
test_df = pd.read_csv(r'C:\Users\ILYA\Downloads\test_sessions.csv (1)\test_sessions.csv', index_col='session_id')
# D:\Загрузки\test_sessions.csv\test_sessions.csv   из красного 
# C:\Users\ILYA\Downloads\test_sessions.csv (1)\test_sessions.csv из серого 

In [4]:
# В обучающей выборке train_sessions.csv:

# Признаки site_i – это индексы посещенных сайтов (расшифровка дана в pickle-файле со словарем site_dic.pkl)
# Признаки time_j – время посещения сайтов site_j
# Целевой признак target – факт того, что сессия принадлежит Элис (то есть что именно Элис ходила по всем этим сайтам)

In [5]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [6]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [7]:
train_df.shape

(253561, 21)

In [8]:
test_df.shape

(82797, 20)

In [9]:
#Создание единого фрейма для удобства разработки фич, без повтора на тестовую выборку отдельно 

In [10]:
#отделить таргет от Трэйна и содеинить Трэйн с Тестом. Сумма строк составит 336 358

In [11]:
DF_Full = pd.concat([train_df.drop('target', axis=1), test_df])

In [12]:
DF_Full.shape

(336358, 20)

In [13]:
DF_Full.head(5)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11


In [159]:
site = ['site%s' % i for i in range(1,11)]
DF_Full2Time = DF_Full.drop(site, axis=1)
for i in range(DF_Full2Time.shape[1]):
    DF_Full2Time.iloc[:,i] = pd.to_datetime(DF_Full2Time.iloc[:,i])

# DF_Full2Time = DF_Full2Time.fillna(pd.Timedelta(seconds=0))
# DF_Full2Time = DF_Full2Time.fillna(0) #.astype(str)
#DF_Full2Time['count_steps'] = (DF_Full2Time == 0).astype(int).sum(axis=1)

# #  pd.to_datetime
# DF_Full2Time['nonzeroval'] = pd.to_datetime(DF_Full2Time.time2) #[year, month, day]
# # apply(lambda x: x[0].timestamp(), axis=1).astype(int)
# # DF_Full2Time['nonzeroval'] = DF_Full2Time.nonzeroval.astype(str)

DF_Full2Time['delta'] = DF_Full2Time.max(axis = 1) - DF_Full2Time.min(axis = 1)
DF_Full2Time

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,delta
session_id,,,,,,,,,,,
1,2014-02-20 10:02:45,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,0 days 00:00:00
2,2014-02-22 11:19:50,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15,2014-02-22 11:20:16,0 days 00:00:26
3,2013-12-16 16:40:17,2013-12-16 16:40:18,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22,2013-12-16 16:40:24,0 days 00:00:07
4,2014-03-28 10:52:12,2014-03-28 10:52:42,2014-03-28 10:53:12,2014-03-28 10:53:42,2014-03-28 10:54:12,2014-03-28 10:54:42,2014-03-28 10:55:12,2014-03-28 10:55:42,2014-03-28 10:56:12,2014-03-28 10:56:42,0 days 00:04:30
5,2014-02-28 10:53:05,2014-02-28 10:55:22,2014-02-28 10:55:22,2014-02-28 10:55:23,2014-02-28 10:55:23,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:57:06,2014-02-28 10:57:11,0 days 00:04:06
...,...,...,...,...,...,...,...,...,...,...,...
82793,2014-10-02 18:20:09,2014-10-02 18:20:09,2014-10-02 18:20:09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,0 days 00:00:00
82794,2014-05-26 14:16:40,2014-05-26 14:16:41,2014-05-26 14:16:44,2014-05-26 14:16:44,2014-05-26 14:17:19,2014-05-26 14:17:19,2014-05-26 14:17:19,2014-05-26 14:17:19,2014-05-26 14:17:19,2014-05-26 14:17:19,0 days 00:00:39
82795,2014-05-02 11:21:56,2014-05-02 11:21:56,2014-05-02 11:21:56,2014-05-02 11:22:03,2014-05-02 11:22:03,2014-05-02 11:22:03,2014-05-02 11:22:03,2014-05-02 11:22:04,2014-05-02 11:22:04,2014-05-02 11:22:04,0 days 00:00:08


In [178]:
#Фичаинжиниринг
time = ['time%s' % i for i in range(1,11)]
DF_Full1 = DF_Full.drop(time, axis=1)

In [180]:
DF_Full1=DF_Full1.fillna(0)
#Число уникальных сайтов в сессии
DF_Full1['num_uniq_sites'] = DF_Full1.apply(pd.Series.nunique, axis=1)-1

#Фича времени посещения первого сайта до полудня или после
DF_Full1['AfterMidday'] = DF_Full.time1.apply(lambda x: x[11:13]).astype(int).apply(lambda x: 1 if x>=12 else 0)

#Число шагов в сессии (Число сайтов - число нулей )
# DF_Full1['count_steps'] = 10-DF_Full2Time['count_steps']

#Длина сессии в минутах 
DF_Full1['SessionLength'] = DF_Full2Time.delta
# apply(pd.Series.nunique, axis=1)-1

#Масштабирование признака по первому моесещению 
DF_Full1['day1'] = DF_Full.time1.apply(lambda x: x[:7].replace('-', '0')).astype(int).apply(lambda x: x/2.013710e+06)


In [162]:
#


In [181]:
DF_Full1.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,num_uniq_sites,AfterMidday,SessionLength,day1
session_id,,,,,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0 days 00:00:00,1.000145
2,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0,6,0,0 days 00:00:26,1.000145
3,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0,3,1,0 days 00:00:07,0.999653
4,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,0,0,0 days 00:04:30,1.000146
5,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0,3,0,0 days 00:04:06,1.000145
6,570,21.0,570.0,21.0,21.0,0.0,0.0,0.0,0.0,0.0,2,1,0 days 00:11:26,1.000146
7,803,23.0,5956.0,17513.0,37.0,21.0,803.0,17514.0,17514.0,17514.0,6,1,0 days 00:01:42,1.000145
8,22,21.0,29.0,5041.0,14422.0,23.0,21.0,5041.0,14421.0,14421.0,6,0,0 days 00:00:06,0.999649
9,668,940.0,942.0,941.0,941.0,942.0,940.0,23.0,21.0,22.0,6,1,0 days 00:00:45,1.000146


In [182]:
DF_Full1 = preprocessing.OneHotEncoder().fit_transform(DF_Full1)

In [183]:
DF_Full1

<336358x185036 sparse matrix of type '<class 'numpy.float64'>'
	with 4709012 stored elements in Compressed Sparse Row format>

In [36]:
#Отсоединение измененной Обучающей выборки от измененной целой и обучение моделей 

In [184]:
X = DF_Full1[:253561]
X

<253561x185036 sparse matrix of type '<class 'numpy.float64'>'
	with 3549854 stored elements in Compressed Sparse Row format>

In [185]:
y = train_df.target

In [186]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=17)

In [187]:
# МОДЕЛИ:

In [188]:
Logistic1 = LogisticRegression(C=1.0, n_jobs=-1)


In [195]:
sgd1 = SGDClassifier(loss='log', random_state=42, n_jobs=-1, alpha=5e-06, max_iter=500)

In [196]:
def Modelauc (model1, model2, X_train, X_test, y_train, y_test):
    pok = []
    model_list = [model1, model2]
    for model in model_list:
        model.fit(X_train, y_train)
        #ab = model.predict_proba(X_test)
        a = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]);
        b = precision_score(y_test, model.predict(X_test));
        c = recall_score(y_test, model.predict(X_test));

        pok.append(model.predict_proba(X_test)[:, 1]);

        print('For model', str(model)[0:11], 'ROC_AUC=', a)
        print('For model', str(model)[0:11], 'Prec=', b)
        print('For model', str(model)[0:11], 'Recall=', c)
        print('____________________________________________________')
        print('                                                    ')
    # return(pok)

    tablet = pd.DataFrame({'aa':pok[0], 'ab': pok[1]})
    tablet['Res'] = (tablet.aa + tablet.ab)/2
    print(tablet)
    print(roc_auc_score(y_test, tablet['Res']))

   

In [197]:
Modelauc(Logistic1, sgd1, X_train, X_test, y_train, y_test)

For model LogisticReg ROC_AUC= 0.9675909574116552
For model LogisticReg Prec= 0.7565217391304347
For model LogisticReg Recall= 0.19908466819221968
____________________________________________________
                                                    
For model SGDClassifi ROC_AUC= 0.9648584436337049
For model SGDClassifi Prec= 0.6834532374100719
For model SGDClassifi Recall= 0.21739130434782608
____________________________________________________
                                                    
             aa        ab       Res
0      0.002236  0.002286  0.002261
1      0.040937  0.068700  0.054819
2      0.000841  0.000740  0.000790
3      0.000012  0.000010  0.000011
4      0.000004  0.000003  0.000003
...         ...       ...       ...
50708  0.000422  0.000300  0.000361
50709  0.000253  0.000054  0.000153
50710  0.000080  0.000095  0.000088
50711  0.000024  0.000030  0.000027
50712  0.000008  0.000016  0.000012

[50713 rows x 3 columns]
0.9667491966086335


In [ ]:
#Отсоединение измененной тестовой выборки от измененной целой

In [192]:
Finial_Test = DF_Full1[253561:]
Finial_Test

<82797x185036 sparse matrix of type '<class 'numpy.float64'>'
	with 1159158 stored elements in Compressed Sparse Row format>

In [193]:
def Result (model1, model2, X_test):
    pok = []
    model_list = [model1, model2]
    for model in model_list:
        model.fit(X, train_df.target)
        
        
        # a = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]);
        # b = precision_score(y_test, model.predict(X_test));
        # c = recall_score(y_test, model.predict(X_test));

        pok.append(model.predict_proba(X_test)[:, 1]);

     
    # return(pok)

    tablet = pd.DataFrame({'aa':pok[0], 'ab': pok[1]})
    tablet['Res'] = (tablet.aa + tablet.ab)/2
    print(tablet)
    print('____________________________________________________')
    print('                                                    ')

    pd.DataFrame({'session_id': range(1, 82798), 'target': tablet.Res}).set_index('session_id').to_csv('Try1_Catch_Alice.csv')

In [194]:
Result(Logistic1, sgd1, Finial_Test)

                 aa        ab       Res
0      1.095949e-04  0.007548  0.003829
1      5.108782e-07  0.000060  0.000030
2      5.895378e-07  0.000106  0.000053
3      2.025562e-06  0.000341  0.000172
4      1.954385e-04  0.018159  0.009177
...             ...       ...       ...
82792  3.040513e-04  0.017883  0.009094
82793  4.789687e-04  0.029352  0.014916
82794  3.128649e-04  0.027252  0.013783
82795  4.338946e-05  0.002861  0.001452
82796  1.881069e-06  0.000249  0.000125

[82797 rows x 3 columns]
____________________________________________________
                                                    


In [116]:
pd.DataFrame({'session_id': range(1, 82798), 'target': gridsearchLOGISTIC2.predict_proba(Finial_Test)[:, 1]}).set_index('session_id').to_csv('Try1_Catch_Alice.csv')